# Level Break Out

### Load the data

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import pandas_ta as ta
import plotly.graph_objects as go
import yfinance as yf


In [301]:
# Specify the stock symbol (NVIDIA in this case) and the date range
stock_symbol = "NVDA"
start_date = "2020-01-01"
end_date = "2023-12-31"

# Fetch stock data from Yahoo Finance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Create DataFrame from the stock data
df = pd.DataFrame(stock_data)

# Filter rows with non-zero volume
#df = df[df['Volume'] != 0]
df.reset_index(inplace=True)  # Resetting index to get timestamp as a column

# Calculate Exponential Moving Average (EMA) with a length of 150
df['EMA'] = ta.ema(df['Close'], length=200)

# Set the timestamp column as the index
#df.set_index('Date', inplace=True)

# Display the DataFrame with timestamps
print(df.tail())

# Select the first 10,000 rows (if available)
df = df.head(10000)



[*********************100%%**********************]  1 of 1 completed
           Date        Open        High         Low       Close   Adj Close  \
1001 2023-12-22  491.950012  493.829987  484.670013  488.299988  488.299988   
1002 2023-12-26  489.679993  496.000000  489.600006  492.790009  492.790009   
1003 2023-12-27  495.109985  496.799988  490.850006  494.170013  494.170013   
1004 2023-12-28  496.429993  498.839996  494.119995  495.220001  495.220001   
1005 2023-12-29  498.130005  499.970001  487.510010  495.220001  495.220001   

        Volume         EMA  
1001  25213900  400.356296  
1002  24420000  401.276035  
1003  23364800  402.200353  
1004  24658700  403.125922  
1005  38869000  404.042281  


In [302]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA
0,2020-01-02,59.687500,59.977501,59.180000,59.977501,59.744045,23753600,NaN
1,2020-01-03,58.775002,59.457500,58.525002,59.017502,58.787781,20538400,NaN
2,2020-01-06,58.080002,59.317501,57.817501,59.264999,59.034317,26263600,NaN
3,2020-01-07,59.549999,60.442501,59.097500,59.982498,59.749027,31485600,NaN
4,2020-01-08,59.939999,60.509998,59.537498,60.095001,59.861092,27710800,NaN
...,...,...,...,...,...,...,...,...
1001,2023-12-22,491.950012,493.829987,484.670013,488.299988,488.299988,25213900,400.356296
1002,2023-12-26,489.679993,496.000000,489.600006,492.790009,492.790009,24420000,401.276035
1003,2023-12-27,495.109985,496.799988,490.850006,494.170013,494.170013,23364800,402.200353
1004,2023-12-28,496.429993,498.839996,494.119995,495.220001,495.220001,24658700,403.125922


In [303]:
#df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
#df=df[df['volume']!=0]
#df.reset_index(drop=True, inplace=True)

#df['EMA'] = ta.ema(df.close, length=150)
#df.tail()

#df=df[0:10000]

#df

### Trend detection

In [304]:
EMAsignal = [0]*len(df)
backcandles = 60

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.EMA[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [305]:
df.head(100)

,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMASignal
0,2020-01-02,59.687500,59.977501,59.180000,59.977501,59.744045,23753600,NaN,0
1,2020-01-03,58.775002,59.457500,58.525002,59.017502,58.787781,20538400,NaN,0
2,2020-01-06,58.080002,59.317501,57.817501,59.264999,59.034317,26263600,NaN,0
3,2020-01-07,59.549999,60.442501,59.097500,59.982498,59.749027,31485600,NaN,0
4,2020-01-08,59.939999,60.509998,59.537498,60.095001,59.861092,27710800,NaN,0
...,...,...,...,...,...,...,...,...,...
95,2020-05-19,87.902496,90.875000,87.627502,88.055000,87.764702,71732000,NaN,3
96,2020-05-20,89.910004,90.430000,88.887497,89.699997,89.404297,58753600,NaN,3
97,2020-05-21,90.514999,90.525002,87.375000,87.752502,87.463219,76106800,NaN,3
98,2020-05-22,88.252502,90.930000,87.132500,90.262497,89.964943,103876400,NaN,3


In [306]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0

    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [316]:
window=3
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [317]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
0,2020-01-02,59.687500,59.977501,59.180000,59.977501,59.744045,23753600,NaN,0,0,NaN,0
1,2020-01-03,58.775002,59.457500,58.525002,59.017502,58.787781,20538400,NaN,0,0,NaN,0
2,2020-01-06,58.080002,59.317501,57.817501,59.264999,59.034317,26263600,NaN,0,0,NaN,0
3,2020-01-07,59.549999,60.442501,59.097500,59.982498,59.749027,31485600,NaN,0,0,NaN,0
4,2020-01-08,59.939999,60.509998,59.537498,60.095001,59.861092,27710800,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1001,2023-12-22,491.950012,493.829987,484.670013,488.299988,488.299988,25213900,400.356296,2,0,NaN,0
1002,2023-12-26,489.679993,496.000000,489.600006,492.790009,492.790009,24420000,401.276035,2,0,NaN,0
1003,2023-12-27,495.109985,496.799988,490.850006,494.170013,494.170013,23364800,402.200353,2,0,NaN,0
1004,2023-12-28,496.429993,498.839996,494.119995,495.220001,495.220001,24658700,403.125922,2,0,NaN,0


In [318]:
def pointpos(x):
    if x['isPivot']==2:
        return x['Low']-1e-3
    elif x['isPivot']==1:
        return x['High']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [319]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
0,2020-01-02,59.687500,59.977501,59.180000,59.977501,59.744045,23753600,NaN,0,0,NaN,0
1,2020-01-03,58.775002,59.457500,58.525002,59.017502,58.787781,20538400,NaN,0,0,NaN,0
2,2020-01-06,58.080002,59.317501,57.817501,59.264999,59.034317,26263600,NaN,0,0,NaN,0
3,2020-01-07,59.549999,60.442501,59.097500,59.982498,59.749027,31485600,NaN,0,0,NaN,0
4,2020-01-08,59.939999,60.509998,59.537498,60.095001,59.861092,27710800,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1001,2023-12-22,491.950012,493.829987,484.670013,488.299988,488.299988,25213900,400.356296,2,0,NaN,0
1002,2023-12-26,489.679993,496.000000,489.600006,492.790009,492.790009,24420000,401.276035,2,0,NaN,0
1003,2023-12-27,495.109985,496.799988,490.850006,494.170013,494.170013,23364800,402.200353,2,0,NaN,0
1004,2023-12-28,496.429993,498.839996,494.119995,495.220001,495.220001,24658700,403.125922,2,0,NaN,0


In [320]:
dfpl = df
#dfpl = df[df['Date'].dt.year == 2020]
#dfpl = df[(df['Date'].dt.year >= 2020) & (df['Date'].dt.year <= 2023)]
fig = go.Figure(data=[go.Candlestick(x=dfpl['Date'],
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl['Date'], y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [329]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    if (candle <= (backcandles+window)) or (candle+window+1 >= len(df)):
        return 0

    localdf = df.iloc[candle-backcandles-window:candle-window] #window must be greater than pivot window to avoid look ahead bias
    highs = localdf[localdf['isPivot'] == 1]['High'].tail(3).values
    lows = localdf[localdf['isPivot'] == 2]['Low'].tail(3).values
    levelbreak = 0
    zone_width = 10
    if len(lows)==3:
        support_condition = True
        mean_low = lows.mean()
        for Low in lows:
            if abs(Low-mean_low)>zone_width:
                support_condition = False
                break
        if support_condition and (mean_low - df.loc[candle].Close)>zone_width*2:
            levelbreak = 1

    if len(highs)==3:
        resistance_condition = True
        mean_high = highs.mean()
        for High in highs:
            if abs(High-mean_high)>zone_width:
                resistance_condition = False
                break
        if resistance_condition and (df.loc[candle].Close-mean_high)>zone_width*2:
            levelbreak = 2
    return levelbreak

In [330]:
#df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=15))
df['pattern_detected'] = df.apply(lambda row: detect_structure(row.name, backcandles=40, window=15), axis=1)

In [331]:
df[df['pattern_detected']!=0]

,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
110,2020-06-10,91.677498,95.000000,91.535004,93.667503,93.401321,63218000,NaN,3,1,95.001000,2
161,2020-08-21,122.017502,128.087494,121.952499,126.834999,126.474586,99986800,NaN,3,0,NaN,2
162,2020-08-24,128.837494,129.125000,125.074997,127.202499,126.841011,49056400,NaN,3,1,129.126000,2
163,2020-08-25,126.307503,127.612503,125.737503,127.500000,127.137680,28907600,NaN,3,0,NaN,2
164,2020-08-26,127.992500,128.684998,126.777496,127.730003,127.367043,32124400,NaN,3,0,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
861,2023-06-05,389.089996,395.649994,387.070007,391.709991,391.602783,39609400,232.758392,2,0,NaN,2
862,2023-06-06,388.299988,391.600006,381.480011,386.540009,386.434204,38872900,234.288558,2,0,NaN,2
863,2023-06-07,389.149994,394.989990,373.559998,374.750000,374.686157,51199800,235.686184,2,2,373.558998,2
864,2023-06-08,377.239990,388.640015,375.049988,385.100006,385.034454,41777200,237.172889,2,0,NaN,2


In [315]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
0,2020-01-02,59.687500,59.977501,59.180000,59.977501,59.744045,23753600,NaN,0,0,NaN,0
1,2020-01-03,58.775002,59.457500,58.525002,59.017502,58.787781,20538400,NaN,0,0,NaN,0
2,2020-01-06,58.080002,59.317501,57.817501,59.264999,59.034317,26263600,NaN,0,0,NaN,0
3,2020-01-07,59.549999,60.442501,59.097500,59.982498,59.749027,31485600,NaN,0,0,NaN,0
4,2020-01-08,59.939999,60.509998,59.537498,60.095001,59.861092,27710800,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1001,2023-12-22,491.950012,493.829987,484.670013,488.299988,488.299988,25213900,400.356296,2,0,NaN,0
1002,2023-12-26,489.679993,496.000000,489.600006,492.790009,492.790009,24420000,401.276035,2,0,NaN,0
1003,2023-12-27,495.109985,496.799988,490.850006,494.170013,494.170013,23364800,402.200353,2,0,NaN,0
1004,2023-12-28,496.429993,498.839996,494.119995,495.220001,495.220001,24658700,403.125922,2,0,NaN,0
